In [3]:
import torch
from nn import *

Prepare encoder

In [4]:
autoencoder = Autoencoder()
autoencoder.load_state_dict(torch.load("./train_output/20230722-141814/best_model.pth"))

encoder_state_dict = {}
for key in autoencoder.state_dict().keys():
    if key.startswith("encoder"):
        encoder_state_dict[key] = autoencoder.state_dict()[key]

encoder = Encoder()
encoder.load_state_dict(encoder_state_dict)

<All keys matched successfully>